# 라이브러리 호출

In [ ]:
pip install segmentation-models-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.5 MB/s 
     |████████████████████████████████| 376 kB 35.6 MB/s 
     |████████████████████████████████| 58 kB 5.2 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12421 sha256=d7f86a15246ef855e601a45b838eba9ef76e4cfa59e14ab40c7018877083ad09
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=217ce786fae463be5109dc27399113a20641c4584fc8fe34dbcc2de4188a4e01
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels


In [ ]:
pip install pytorch-msssim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
 import os, tqdm, math, random, argparse

import numpy as np
import pandas as pd

from glob import glob
from PIL import Image
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader, Dataset

from tqdm.notebook import tqdm

import segmentation_models_pytorch as smp
from pytorch_msssim import MS_SSIM

import cv2
import random

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
print(device)

#device는 학습 환경을 cpu로 할 것인지 아니면 gpu로 할것인지 에 대한 변수이다.
# torch.cuda.is_available()는 말 그대로 cuda(gpu)을 사용할 수 있으면 사용하겠다는 의미

cpu


# directory 확인

In [ ]:
train_csv = pd.read_csv('/content/drive/MyDrive/train.csv')
test_csv = pd.read_csv('/content/drive/MyDrive/test.csv')

In [ ]:
train_csv.head()

In [ ]:
test_csv.head()

In [ ]:
train_all_input_files = '/content/drive/MyDrive/train/'+train_csv['input_img']
train_all_label_files = '/content/drive/MyDrive/label/'+train_csv['label_img']
test_all_input_files = '/content/drive/MyDrive/test/'+test_csv['input_img']

# 학습 데이터와 검증 데이터를 분리

In [ ]:
train_input_files = train_all_input_files[60:].to_numpy()
train_label_files = train_all_label_files[60:].to_numpy()

In [ ]:
vaild_input_files = train_all_input_files[:60].to_numpy()
vaild_label_files = train_all_label_files[:60].to_numpy()

In [ ]:
train_input_files

In [ ]:
vaild_input_files

# 빛번짐 데이터 열어서 확인

In [ ]:
for i in range(4):
    origin_img = Image.open(f'/content/drive/MyDrive/train/train_input_{i+10060}.png')
    trans_img = Image.open(f'/content/drive/MyDrive/label/train_label_{i+10060}.png')
    fix , ax = plt.subplots(ncols = 2,figsize = (8,8))
    ax[0].set_title('input')
    ax[0].imshow(origin_img)
    ax[1].set_title('label')
    ax[1].imshow(trans_img)
    plt.show()

# 빛번짐 최소화 데이터 처리

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, label,w,h ,is_train=True):
        self.data  = data
        self.label = label
        self.w = w
        self.h = h
        self.is_train = is_train
        
    def __len__(self):
        return len(self.data)
    
    def transform(self, image, label):
        
        resizer = transforms.Resize(size=(self.h, self.w))
        image = resizer(image)
        label = resizer(label)

        image = TF.to_tensor(image)
        label = TF.to_tensor(label)
        return image, label
    
    def __getitem__(self, idx):
        origin_img = Image.open(self.data[idx])
        label_img  = Image.open(self.label[idx])
        
        origin_img, label_img = self.transform(origin_img, label_img)
        return origin_img, label_img
    
#def __init__는 생성자
#def __len__은 데이셋의 길이를 변환
#def __transform 에서 resize = 이미지 크기 재지정 , to_tensor = 이미지를 tensor형으로 변경
#def __getitem__은 이미지를 호출하고 transform함수를 적용 시킨 후 이미지를 호출

In [ ]:
class testDataset(Dataset):
    def __init__(self, data,is_train=True):
        self.data  = data
        self.is_train = is_train
        
    def __len__(self):
        return len(self.data)
    
    def transform(self, image):
        
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(size=(384, 512))
        ])
        image = transform(image)
        
        return image
    
    def __getitem__(self, idx):
        origin_img = Image.open(self.data[idx])
        
        origin_img = self.transform(origin_img)
        return origin_img

In [ ]:
train_dataset = CustomDataset(train_input_files, train_label_files,512,384)
vaild_dataset = CustomDataset(vaild_input_files, vaild_label_files,512,384)
test_dataset = testDataset(test_all_input_files)

In [ ]:
for i in range(4):
    origin_img = Image.open(f'/content/drive/MyDrive/train/train_input_{i+10060}.png')
    trans_img = train_dataset[i][0].numpy().transpose(1,2,0)
    fix , ax = plt.subplots(ncols = 2,figsize = (8,8))
    ax[0].set_title('before reduce')
    ax[0].imshow(origin_img)
    ax[1].set_title('after reduce')
    ax[1].imshow(trans_img)
    plt.show()

In [ ]:
train_loader = DataLoader(train_dataset , batch_size = 4, shuffle = True)
vaild_loader = DataLoader(vaild_dataset , batch_size = 1, shuffle = False)
test_loader = DataLoader(test_dataset , batch_size = 4 , shuffle = True )

# 빛번짐 최소화 인공지능 모델 선정

In [ ]:
model = smp.UnetPlusPlus(encoder_name='resnet18',
                encoder_weights='imagenet',
                in_channels=3, classes=3, activation='sigmoid')

'''
model = smp.UnetPlusPlus(encoder_name='efficientnet-b4',
                encoder_weights='imagenet',
                in_channels=3, classes=3, activation='sigmoid')
                

model = smp.UnetPlusPlus(encoder_name='mobilenet_v2',
                encoder_weights='imagenet',
                in_channels=3, classes=3, activation='sigmoid')
'''

In [ ]:
model.to(device)
model
#model.load_state_dict(torch.load('/content/drive/MyDrive/efficientnet.pt'))

# 빛번짐 최소화 인공지능 모델 학습

In [ ]:
criterion = torch.nn.MSELoss().to(device) #예상 답과 실제 답의 차이를 줄이기 위한 함수
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #가중치를 갱신해주는 함수
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size = 10, gamma=0.8)#학습에 사용 되는 가중치를 조절해주는 함수

In [ ]:
train_loss = []
vaild_loss=[]
#model.load_state_dict(torch,load('/content/drive/MyDrive/efficientnet.pt'))
for epoch in range(300):
    total_loss = 0
    model.train()
    
    for image , label in tqdm(train_loader,total=len(train_loader),leave = False):
        image = image.to(device)
        label = label.to(device)
        output = model(image.float())
        loss = torch.sqrt(criterion(output,label))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    train_loss.append(total_loss/len(train_loader))
    torch.save(model.state_dict(), '/content/drive/MyDrive/efficientnet.pt')
    print("epoch :" , epoch+1 , "loss :" , total_loss/len(train_loader))
    
    scheduler.step()
    
    idx = 1
    '''
    model.eval()
    with torch.no_grad():
        for image , label in tqdm(vaild_loader,total=len(vaild_loader),leave = False):
            image = image.to(device)
            label = label.to(device)
            output = model(image.float())
            loss = torch.sqrt(criterion(output,label))
            
            total_loss += loss.item()
            

            target_path = 'data/result/' + str(epoch+1)+'/'
            if not os.path.exists(target_path):
                os.makedirs(target_path, exist_ok=True)

            grid = torchvision.utils.make_grid(tensor=torch.vstack([image,label,output]), nrow=image.shape[0])    # Saving results
            torchvision.utils.save_image(grid, target_path+str(idx)+'.png')
            idx +=1
        vaild_loss.append(total_loss/len(vaild_loader))
        print("val_epoch :" , epoch+1 , "val_loss :" , total_loss/len(vaild_loader))
    '''

#요약으로 dataset에서 문제(image)와 답지(label)을 호출 하고 model에 문제를 넣음(output = modle(image))
#output은 예상 답을 의미 하고 loss변수는 예상 답과 실제 답에 대한 차이를 줄여 나감
#zero_grad는 가중치를 초기화해주는 함수
#loss backward는 역전파 알고리즘에 대한 것으로 신경망 방향을 순방향으로 한번 역방향으로 한번 확인 후 비교를 하는 것
    

# 빛번짐 모델 검증

In [ ]:
model.eval()
model.load_state_dict(torch.load( '/content/drive/MyDrive/efficientnet.pt'))
for image,label in vaild_loader:
    recon = model(image.to(device))
    _, ax = plt.subplots(1, 3, figsize=(15,15))
    ax[0].set_title('input')
    ax[0].imshow(np.moveaxis(image.reshape(3,384,512).cpu().numpy(),0,2))
    ax[1].set_title('label')
    ax[1].imshow(np.moveaxis(label.reshape(3,384,512).cpu().numpy(),0,2))
    ax[2].set_title('predict')
    ax[2].imshow(np.moveaxis(recon.reshape(3,384,512).cpu().detach().numpy(),0,2))

# 밤낮 데이터 파일 크기 및 확인

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
origin_img = Image.open(f'/content/drive/MyDrive/밤사진/795378f0-db2d80b2.jpg')
trans_img = Image.open(f'/content/drive/MyDrive/낮사진/58610009-f3b500dd.jpg')
fix , ax = plt.subplots(ncols = 2,figsize = (8,8))
ax[0].set_title('input(night)')
ax[0].imshow(origin_img)
ax[1].set_title('label(day)')
ax[1].imshow(trans_img)
plt.show()

# 밤낮 전환 인공지능 데이터 처리

In [ ]:
class Dataset(object):
    def __init__(self, label_dir, input_dir, image_size, scale):
        self.label_dir = [os.path.join(label_dir, x) for x in os.listdir(label_dir) if self.check_image_file(x)]
        self.input_dir = [os.path.join(input_dir, x) for x in os.listdir(input_dir) if self.check_image_file(x)]
        self.image_size = image_size
        self.to_Tensor = transforms.ToTensor()
        self.resize = transforms.Resize((128 , 128 ), interpolation=Image.BICUBIC)
        self.rotates = [0, 90, 180, 270]
    
    def check_image_file(self, filename: str):
        return any(filename.endswith(extension) for extension in [".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".JPG", ".JPEG", ".PNG", ".BMP"])
    
    def data_augmentation(self, hr, lr):

        width, height = hr.size
        
        hr = hr.resize((512, 384), resample=Image.BICUBIC)
        lr = lr.resize((512, 384), resample=Image.BICUBIC)
    
        return hr, lr

    def __getitem__(self, idx):
    
    
        hr = Image.open(self.label_dir[idx]).convert("RGB")
        lr = Image.open(self.input_dir[idx]).convert("RGB") 

        hr, lr = self.data_augmentation(hr, lr) 
        
        return self.to_Tensor(hr), self.to_Tensor(lr) 

    def __len__(self):
        return len(self.label_dir)

In [ ]:
class TestDataset(object):
    def __init__(self, input_dir, image_size, scale):
        self.input_dir = [os.path.join(input_dir, x) for x in os.listdir(input_dir) if self.check_image_file(x)]
        self.image_size = image_size
        self.to_Tensor = transforms.ToTensor()
        self.resize = transforms.Resize((image_size , image_size ), interpolation=Image.BICUBIC)
        self.rotates = [0, 90, 180, 270]
     
    
    def check_image_file(self, filename: str):
        return any(filename.endswith(extension) for extension in [".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".JPG", ".JPEG", ".PNG", ".BMP"])
    

    def data_augmentation(self, hr):

        width, height = hr.size

        hr = hr.resize((512, 384), resample=Image.BICUBIC) # 테스트용은 밤 하나이기 때문.

        return hr

    def __getitem__(self, idx):

        hr = Image.open(self.input_dir[idx]).convert("RGB") 

        hr = self.data_augmentation(hr) 
        return self.to_Tensor(hr)

    def __len__(self):
        return len(self.input_dir)

In [ ]:
train_dir = '/content/drive/MyDrive/밤사진/' # 밤
label_dir = '/content/drive/MyDrive/낮사진/' # 낮
test_dir = '/content/drive/MyDrive/test/' #테스트 값.

In [ ]:
train_dataset = Dataset(train_dir , label_dir,256,1)
test_dataset = TestDataset(test_dir , 256,1)

In [ ]:
pwd

In [ ]:
origin_img = Image.open(f'/content/drive/MyDrive/밤사진/795378f0-db2d80b2.jpg')
trans_img = train_dataset[1][0].numpy().transpose(1,2,0)
fix , ax = plt.subplots(ncols = 2,figsize = (8,8))
ax[0].set_title('before reduce')
ax[0].imshow(origin_img)
ax[1].set_title('after reduce')
ax[1].imshow(trans_img)
plt.show()

In [ ]:
origin_img = Image.open(f'/content/drive/MyDrive/낮사진/58610009-f3b500dd.jpg')
trans_img = train_dataset[0][1].numpy().transpose(1,2,0)
fix , ax = plt.subplots(ncols = 2,figsize = (8,8))
ax[0].set_title('before reduce')
ax[0].imshow(origin_img)
ax[1].set_title('after reduce')
ax[1].imshow(trans_img)
plt.show()

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size = 2,
    shuffle = True,
)

test_loader = DataLoader(
    test_dataset,
    batch_size = 3,
    shuffle = False
)

# generator

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.ReflectionPad2d(1), # reflectionpadding이란 가장자리 기준으로 input값을 패딩영역에 반전하여 복사하여 채우는 기법. 여기선 가장자리 1줄을 채움.
            nn.Conv2d(in_channels, in_channels, 3), #CNN.
            nn.InstanceNorm2d(in_channels), #BatchNorm과 다른 점은 Batch는 전체 Dataset기준으로 Batch를 Normalize하는 것이라면, Instance는 mini-Batch단위로 instance들을 Normalize한다는 점.
            nn.GELU(), # dropout, zoneout, ReLu 함수의 특성을 조합한 것이 GeLU, ReLU가 뛰어나긴 하나 음수가 되어버리면 그 때의 모든 기울기는 0이 되어버리기에 음수에서 조금의 기울기를 주는 형태.
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_channels, in_channels, 3),
            nn.InstanceNorm2d(in_channels)
        )
    
    def forward(self, x):  # weight를 통과한 layer들과 통과하지 않은 값들을 더한 residual mapping방식. 
        return x + self.block(x) # x는 통과하지 않은 값들, self.block(x)는 통과한 값들.

In [ ]:
class GeneratorResNet(nn.Module):
    def __init__(self, in_channels, num_residual_blocks=3):  # 이 클래스에서 resiudalblock 클래스를 이용할 때 쓰이는 변수. 현재는 layer 9번 쓴다고 선언 되어있고 block수는 맘대로 선언해도 됨.
        super(GeneratorResNet, self).__init__()
        
        out_channels=64   #최초의 convolution 블록. 처음 출력값은 64개.
        self.conv = nn.Sequential(
            nn.ReflectionPad2d(in_channels), 
            nn.Conv2d(in_channels, out_channels, 2*in_channels+1),# in_channels = 3 이유는 이미지는 모두 rgb를 가지고 있기 때문.
            nn.InstanceNorm2d(out_channels),
            nn.GELU(),
        )

        channels = out_channels # convolution 후 나온 값을 저장한 모습. 즉, 64가 저장 된다.
                                
                                # 이 작업으로 64개의 출력값을 가지고 있는 은닉층 형성.
        
        self.down = [] #데이터 수를 줄이는게 아니라 우리 빛번짐에서 이미지 축소하고 확장하잖아? 그중 축소하면서 특징 추출하는 과정임.
                       #generator는 기본적으로 Unet model을 기본으로 만들기 때문에 그 중 이미지 축소하면서 특징 추출하는거지
                       
        for _ in range(2): # 2번 반복하는데 index부분이 필요가 없으므로 _ 사용.
            out_channels = channels * 2 #채널 값을 2배로 늘린 후 저장(첫 컨볼루션에서 3-64 여기서 64-128, 그 후 128-256)
            self.down += [                                                  # 이미지 축소 과정.
                nn.Conv2d(channels, out_channels, 3, stride=2, padding=1),
                nn.InstanceNorm2d(out_channels),
                nn.GELU(),
            ]
            channels = out_channels # 각각의 출력값을 저장. 1번째 convolution에선 128. 2번째 convolution에선 256저장.
        self.down = nn.Sequential(*self.down) #위에 for문 과정 down 리스트에 저장.
        
        self.trans = [ResidualBlock(channels) for _ in range(num_residual_blocks)]  #residual block을 돌리는 코드.
                                                                                    # for문을 풀자면 ResidualBlock클래스를 9번 돌리고 그 결괏값을 trans에 저장한다는 코드.
                                                                                    
        self.trans = nn.Sequential(*self.trans) #trans라는 리스트에 resudial block 9개층을 저장(여기도 모델에 resudial block을 몇을 주냐에 따라 층 개수 달라짐)
        self.up = [] # residual block을 거쳐서 mapping이 진행된 값을 저장.
        
        for _ in range(2): #upsampling과정. 여기서는 이미지를 축소했기 때문에 다시 복원하는 과정이다.
            out_channels = channels // 2 # 컨볼루션 과정에서 3-64-128-256형태로 축소했기 때문에 다시 256-128-64-3형태로 가야한다.
                                         #여기서는 이미지를 다시 확장하는 과정임
            self.up += [
                nn.Upsample(scale_factor=2), # 이미지가 만약 2*2면 scale_factor = 2면 이미지가 4*4가 됨 이렇게 확장하는과정에서 빈공간의 수를 어떻게 채울지에 따라 3가지 방식 여기선 bil...어쩌고
                nn.Conv2d(channels, out_channels, 3, stride=1, padding=1), 
                nn.InstanceNorm2d(out_channels),
                nn.GELU(),
            ]
            channels = out_channels #위에 설명 해줬으니까 알겠지? / 다시 64를 저장.
        self.up = nn.Sequential(*self.up) #up list에 이미지 확장해주는 layer를 리스트로 저장
        
        self.out = nn.Sequential( #출력 layer. 이제 64-3 convolution 진행.
            nn.ReflectionPad2d(in_channels),
            nn.Conv2d(channels, in_channels, 2*in_channels+1),
            nn.Tanh() #마지막은 Sigmoid나 tanh둘중에 뭐쓸지 고민중 
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = self.down(x)
        x = self.trans(x)
        x = self.up(x)
        x = self.out(x)
        return x

# discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_channels):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(
            *self.block(in_channels, 32, normalize=False),
            *self.block(32, 64),  
            *self.block(64, 128), 
            *self.block(128, 256),
            
            nn.ZeroPad2d((1,0,1,0)), 
            nn.Conv2d(256, 1, 4, padding=1)
        )
        
        self.scale_factor = 16
        self.m = nn.Sigmoid()
    
    @staticmethod
    def block(in_channels, out_channels, normalize=True):
        layers = [nn.Conv2d(in_channels, out_channels, 4, stride=2, padding=1)]
        if normalize:
            layers.append(nn.InstanceNorm2d(out_channels))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        
        return layers
        
    def forward(self, x):
        x = self.model(x)
        x = self.m(x)
        return x
    

In [ ]:
G_AB = GeneratorResNet(3, num_residual_blocks=9)
D_B = Discriminator(3)

G_BA = GeneratorResNet(3, num_residual_blocks=9)
D_A = Discriminator(3)

In [ ]:
criterion_GAN = nn.BCELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.MSELoss()

In [ ]:
cuda = torch.cuda.is_available()
print(f'cuda: {cuda}')
if cuda:
    G_AB = G_AB.cuda()
    #G_AB.load_state_dict(torch.load('/content/drive/MyDrive/weight/'+'G_AB2.pt'))
    D_B = D_B.cuda()
    #D_B.load_state_dict(torch.load('/content/drive/MyDrive/weight/'+'D_B2.pt'))
    G_BA = G_BA.cuda()
    #G_BA.load_state_dict(torch.load('/content/drive/MyDrive/weight/'+'G_BA2.pt'))
    D_A = D_A.cuda()
    #D_A.load_state_dict(torch.load('/content/drive/MyDrive/weight/'+'D_A2.pt'))
    
    criterion_GAN = criterion_GAN.cuda()
    criterion_cycle = criterion_cycle.cuda()
    criterion_identity = criterion_identity.cuda()

# genrator 결과 시각화

In [ ]:
from torchvision.utils import make_grid
import random
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor

def sample_images(real_A, figside=1.5):
    G_AB.cuda()
    G_AB.eval()
    G_BA.eval()
    
    real_A = real_A.type(Tensor) #진짜 밤 사진.
    fake_B = G_AB(real_A).detach()
    reconv_A = G_BA(fake_B).detach()
    '''
    real_B = real_B.type(Tensor)
    fake_A = G_BA(real_B).detach()
'''
    
    nrows = real_A.size(0)
    real_A = make_grid(real_A, nrow=nrows, normalize=True)
    fake_B = make_grid(fake_B, nrow=nrows, normalize=True)
    reconv_A = make_grid(reconv_A, nrow=nrows, normalize=True)
    #fake_A = make_grid(fake_A, nrow=nrows, normalize=True)
    
    image_grid = torch.cat((real_A, fake_B, reconv_A), 1).cpu().permute(1, 2, 0)
    
    plt.figure(figsize=(15, 10))
    plt.imshow(image_grid)
    plt.axis('off')
    plt.show()

In [ ]:
real_A = next(iter(test_loader))
print(real_A.shape)
sample_images(real_A)

# 밤낮 전환 인공지능 인공지능 모델 학습

In [ ]:
criterion_GAN = nn.BCELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.MSELoss()

In [ ]:
import itertools
lr = 0.0002


optimizer_G = torch.optim.Adam(
    itertools.chain(G_AB.parameters(), G_BA.parameters()), lr=lr , betas = (0.5,0.999)
)

optimizer_D_A = torch.optim.Adam(
    D_A.parameters(), lr=lr , betas = (0.5,0.999)
)

optimizer_D_B = torch.optim.Adam(
    D_B.parameters(), lr=lr , betas = (0.5,0.999)
)

In [ ]:
n_epoches = 300
decay_epoch = 20

lambda_func = lambda epoch: 1 - max(0, epoch-decay_epoch)/(n_epoches-decay_epoch)

lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=lambda_func)
lr_scheduler_D_A = torch.optim.lr_scheduler.LambdaLR(optimizer_D_A, lr_lambda=lambda_func)
lr_scheduler_D_B = torch.optim.lr_scheduler.LambdaLR(optimizer_D_B, lr_lambda=lambda_func)

In [ ]:
'''
torch.manual_seed(777)

import gc
from tqdm.notebook import tqdm

for epoch in range(100):
    gc.collect()
    torch.cuda.empty_cache()

    for i, (real_A, real_B) in tqdm(enumerate(train_loader),total = len(train_loader)):
        real_A, real_B = real_A.type(Tensor), real_B.type(Tensor)
        
        # groud truth
        out_shape = [real_A.size(0), 1, real_A.size(2)//D_A.scale_factor, real_A.size(3)//D_A.scale_factor]
        valid = torch.ones(out_shape).type(Tensor)
        fake = torch.zeros(out_shape).type(Tensor)
        
        G_AB.train()
        G_BA.train()
        D_A.train()
        D_B.train()
        
        optimizer_G.zero_grad()
        
        fake_B = G_AB(real_A) 
        fake_A = G_BA(real_B)  
        
        loss_id_A = torch.sqrt(criterion_identity(fake_B, real_A)) 
        loss_id_B = torch.sqrt(criterion_identity(fake_A, real_B))
        loss_identity = (loss_id_A + loss_id_B) / 2

        loss_GAN_AB = criterion_GAN(D_B(fake_B),valid)
        loss_GAN_BA = criterion_GAN(D_A(fake_A), valid)
        loss_GAN = (loss_GAN_AB + loss_GAN_BA) / 2
        
        recov_A = G_BA(fake_B)
        recov_B = G_AB(fake_A)
        
        loss_cycle_A = criterion_cycle(recov_A, real_A)
        loss_cycle_B = criterion_cycle(recov_B, real_B)
        loss_cycle = (loss_cycle_A + loss_cycle_B) / 2
        
        loss_G = 5.0 *loss_identity + loss_GAN + 10.0*loss_cycle
        
        loss_G.backward()
        optimizer_G.step()
        
        optimizer_D_A.zero_grad()
        
        loss_real = criterion_GAN(D_A(real_A), valid)
        loss_fake = criterion_GAN(D_A(fake_A.detach()), fake)
        loss_D_A = (loss_real + loss_fake) / 2
        
        loss_D_A.backward()
        optimizer_D_A.step()
        
        optimizer_D_B.zero_grad()
        
        loss_real = criterion_GAN(D_B(real_B), valid)
        loss_fake = criterion_GAN(D_B(fake_B.detach()), fake)
        loss_D_B = (loss_real + loss_fake) / 2
        
        loss_D_B.backward()
        optimizer_D_B.step()
        
    
        torch.save(G_AB.state_dict(), 'data/weight/'+f'G_AB{epoch+1}.pt') # model.load_state_dict(torch.load(G_AB2.pth))
        torch.save(G_BA.state_dict(), 'data/weight/'+f'G_BA{epoch+1}.pt')
        torch.save(D_A.state_dict(), 'data/weight/'+f'D_A{epoch+1}.pt')
        torch.save(D_B.state_dict(), 'data/weight/'+f'D_B{epoch+1}.pt')
            
    lr_scheduler_G.step()
    lr_scheduler_D_A.step()
    lr_scheduler_D_B.step()
    
 
    test_real_A = next(iter(test_loader))
    if (epoch+1) % 5 == 0:
        sample_images(test_real_A)

    loss_D = (loss_D_A + loss_D_B) / 2
    if (epoch+1) % 10 == 0:
        print(f'[Epoch {epoch+1}/{n_epoches}]')
        print(f'[G loss: {loss_G.item()} | identity: {loss_identity.item()} GAN: {loss_GAN.item()} cycle: {loss_cycle.item()}]')
        print(f'[D loss: {loss_D.item()} | D_A: {loss_D_A.item()} D_B: {loss_D_B.item()}]') 
        
        '''

In [ ]:
!nvidia-smi

# 밤낮 인공지능 검증

In [ ]:
real_A= next(iter(test_loader))
print(real_A.shape)

sample_images(real_A)

# 전체 결과 보기(빛 줄인 후 밤낮 전환)

In [ ]:
model.eval()
G_AB.eval()
model.load_state_dict(torch.load('/content/drive/MyDrive/efficientnet.pt'))
G_AB.load_state_dict(torch.load('/content/drive/MyDrive/밤.pt'))
with torch.no_grad():
    for data  in test_loader:
        data = data
        predict = model(data.to(device))  
        pred = G_AB(predict.to(device))
        #print(image.shape , output.shape ,out.shape)
        break

In [ ]:
for i in range(2):
    
    fix , ax = plt.subplots(ncols = 3,figsize = (20,15))
    ax[0].set_title('input(vaildation)')
    ax[0].imshow(np.moveaxis(data[i].cpu().numpy(),0,2))
    ax[1].set_title('reduce light')
    ax[1].imshow(np.moveaxis(predict[i].cpu().numpy(),0,2))
    ax[2].set_title('transform to day')
    ax[2].imshow(np.moveaxis(pred[i].cpu().numpy(),0,2))

# 전체 결과 보기(밤낮 전환 후 빛 줄이기)

In [ ]:
model.eval()
G_AB.eval()
model.load_state_dict(torch.load('/content/drive/MyDrive/efficientnet.pt'))
G_AB.load_state_dict(torch.load('/content/drive/MyDrive/밤.pt'))
with torch.no_grad():
    for image  in test_loader:
        image = image
        output = G_AB(image.to(device))  
        out = model(output.to(device))
        print(image.shape , output.shape ,out.shape)
        break

In [ ]:
for i in range(2):
    
    fix , ax = plt.subplots(ncols = 3,figsize = (20,15))
    ax[0].set_title('input(vaildation)')
    ax[0].imshow(np.moveaxis(image[i].cpu().numpy(),0,2))
    ax[1].set_title('transform to day')
    ax[1].imshow(np.moveaxis(output[i].cpu().numpy(),0,2))
    ax[2].set_title('reduce light')
    ax[2].imshow(np.moveaxis(out[i].cpu().numpy(),0,2))

# 그냥 밤낮만 전환 했을 때랑 빛 최소화 후 밤낮 전환 했을 때랑 밤낮 전환 후 빛 최소화를 비교

In [ ]:
for i in range(3):
    
    fix , ax = plt.subplots(ncols = 4,figsize = (20,15))
    ax[0].set_title('input(vaildation)')
    ax[0].imshow(np.moveaxis(image[i].cpu().numpy(),0,2))
    ax[1].set_title('only transform night to day')
    ax[1].imshow(np.moveaxis(output[i].cpu().numpy(),0,2))
    ax[2].set_title('reduce light and transform night to day')
    ax[2].imshow(np.moveaxis(pred[i].cpu().numpy(),0,2))
    ax[3].set_title('transform night to day and reduce light')
    ax[3].imshow(np.moveaxis(out[i].cpu().numpy(),0,2))